In [ ]:
import os
import ssl
from pyesgf.logon import LogonManager
from pyesgf.search import SearchConnection

In [10]:
# define your query, eg:
query = {
    'project': 'CMIP6',
    'activity_drs': 'ScenarioMIP',
    'experiment_id': 'ssp534-over',
    'variable': 'tas',
    'frequency': 'mon'
}

# ensure the following are saved as environment variables
USERNAME = os.environ['ESGF_USERNAME']
PASSWORD = os.environ['ESGF_PASSWORD']
DATA_PATH = os.environ['DATA_HOME']

In [11]:
# check ESGF for number of datasets that satisfy query
conn = SearchConnection('http://esgf-data.dkrz.de/esg-search', distrib=True)
context = conn.new_context(**query, facets=query.keys())
context.hit_count

61

In [12]:
# login to ESGF and generate SSL context
myproxy_host = 'esgf-node.ipsl.upmc.fr'

lm = LogonManager()
lm.logon(username=USERNAME, password=PASSWORD, hostname=myproxy_host)

sslcontext = ssl.create_default_context(purpose=ssl.Purpose.SERVER_AUTH)
sslcontext.load_verify_locations(capath=lm.esgf_certs_dir)
sslcontext.load_cert_chain(lm.esgf_credentials)

# check if logged in
lm.is_logged_on()

True

In [13]:
# generate results and check an example dataset to verify all is working as expected
results = context.search()
example_dataset = results[0]
example_dataset.dataset_id

'CMIP6.ScenarioMIP.NCC.NorESM2-LM.ssp534-over.r1i1p1f1.Amon.tas.gn.v20210811|esgf.ceda.ac.uk'

In [14]:
# and now an example file within that dataset, including its http download link
example_files = example_dataset.file_context().search(ignore_facet_check=True)
example_file = example_files[0]
example_file.download_url

'https://esgf.ceda.ac.uk/thredds/fileServer/esg_cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ssp534-over/r1i1p1f1/Amon/tas/gn/v20210811/tas_Amon_NorESM2-LM_ssp534-over_r1i1p1f1_gn_204001-204912.nc'